<a href="https://colab.research.google.com/github/J3rmed/ai4eng_2025_2/blob/main/04%20-%20modelo%20con%20preprocesado%20y%20RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 03 - modelo con Random Forest

Un modelo utilizando Random Forest. Seguiendo el siguiente flujo de trabajo: cargar los datos, aplicar el preprocesado, evaluar y entrenar el modelo, finalmente, procesar los datos de prueba y generar un archivo de submission para Kaggle.

In [1]:

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import warnings

# Importante: Cargar el kaggle token
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
!unzip udea*.zip > /dev/null

replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [4]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  59185238 test.csv
   692501  10666231 143732437 train.csv
  1286075  15528571 207634348 total


In [5]:
z_train_raw = pd.read_csv("train.csv")
z_test_raw = pd.read_csv("test.csv")

print("Datos de entrenamiento cargados:", z_train_raw.shape)
print("Datos de prueba cargados:", z_test_raw.shape)

Datos de entrenamiento cargados: (692500, 21)
Datos de prueba cargados: (296786, 20)


In [6]:
# Definición de Columnas
target_col = 'RENDIMIENTO_GLOBAL'
columnas_a_eliminar = ['F_TIENEINTERNET.1', 'E_PRIVADO_LIBERTAD']

X = z_train_raw.drop(columns=[target_col] + columnas_a_eliminar)
y = z_train_raw[target_col]

numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.drop('ID').tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

# Codificación de la Variable Objetivo
mapa_rendimiento = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
y_encoded = y.map(mapa_rendimiento)

print("Preprocesamiento definido.")

Preprocesamiento definido.


In [7]:
# Creación del Pipeline Final (Preprocesador + Random Forest)

rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=200,
        max_depth=15,
        random_state=42,
        n_jobs=-1
    ))
])

print("Entrenando el modelo Random Forest con todos los datos de entrenamiento...")

# Entrenamiento
rf_pipeline.fit(X, y_encoded)

print("Entrenamiento finalizado.")

Entrenando el modelo Random Forest con todos los datos de entrenamiento...
Entrenamiento finalizado.


In [8]:
print("Procesando datos de prueba y generando predicciones...")

X_test = z_test_raw.drop(columns=columnas_a_eliminar, errors='ignore')
test_ids = X_test['ID']

preds_test_encoded = rf_pipeline.predict(X_test)

mapa_inverso_rendimiento = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}
preds_test_texto = [mapa_inverso_rendimiento[p] for p in preds_test_encoded]

submission = pd.DataFrame({'ID': test_ids, 'RENDIMIENTO_GLOBAL': preds_test_texto})

submission.to_csv("submission_rf.csv", index=False)

print("Archivo 'submission_rf.csv' generado exitosamente.")
submission.head()

Procesando datos de prueba y generando predicciones...
Archivo 'submission_rf.csv' generado exitosamente.


,ID,RENDIMIENTO_GLOBAL
0,550236,alto
1,98545,medio-alto
2,499179,alto
3,782980,bajo
4,785185,medio-bajo
